# QDRANTを用いたvectorサーチのデモ

1. 技術ブログの検索
  - 決まったフォーマットで出力
  - URLも同時に返却
  - 上位3つくらい

2. 技術ブログの登録


# 方針

- LangChain.Agentを利用せずに、LLamaIndex単体で実装

In [1]:
import json

from dotenv import load_dotenv
from llama_index import SimpleWebPageReader, LLMPredictor, ServiceContext, GPTQdrantIndex, OpenAIEmbedding
from llama_index.prompts.prompts import QuestionAnswerPrompt, RefinePrompt
import qdrant_client
from langchain import OpenAI

COLLECTION_NAME = "chatgpt_search_collection"
HOST = "localhost"
PORT = 6333
PREDICTOR_MODEL_NAME = "gpt-3.5-turbo"
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
INITIAL_URLS = [
    "https://dev.classmethod.jp/articles/lang-chain-agent-customized-by-llama-index-tool/",
    "https://yukoishizaki.hatenablog.com/entry/2020/05/24/145155",
    "https://runble1.com/gcp-terraform-cloud-run/"
]
ADDITIONAL_URL = "https://zenn.dev/tfutada/articles/acf8adbb2ba5be"

# カスタムテンプレートの作成
CUSTOM_TEXT_QA_PROMPT_TMPL = (
    "コンテキストは以下です. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "コンテキストが与えられた場合, "
    "質問に回答してください: {query_str}\n"
)
CUSTOM_TEXT_QA_PROMPT = QuestionAnswerPrompt(CUSTOM_TEXT_QA_PROMPT_TMPL)

CUSTOM_REFINE_PROMPT_TMPL = (
    "元の質問: {query_str}\n"
    "オリジナルの回答: {existing_answer}\n"
    "以下のコンテキストを使って、オリジナルの回答を推敲することができます.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "コンテキストを元に、オリジナルの回答を、より元の質問に沿ったものに推敲してください. "
    "もしコンテキストが有用なものでなければ、オリジナルの回答を返却してください"
)
CUSTOM_REFINE_PROMPT = RefinePrompt(CUSTOM_REFINE_PROMPT_TMPL)

# 環境変数の読み込み
load_dotenv('../.env')

True

In [ ]:
# デバッグする際に実行
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Indexの構築 & 検索

In [2]:
# clientの作成
client = qdrant_client.QdrantClient(
    host=HOST,
    https=False,
    port=PORT
)

In [4]:
# client.get_collection(collection_name=COLLECTION_NAME)

In [5]:
# webページからdocumentクラスを作成
documents = SimpleWebPageReader(html_to_text=True).load_data(INITIAL_URLS)

# extra_infoにurlを追加
for document, url in zip(documents, INITIAL_URLS):
    document.extra_info = {"url": url}

In [6]:
# 既にDBにcollectionが存在する場合、その内容を利用
try:
    client.get_collection(collection_name=COLLECTION_NAME)
    initial_documents = []
except:
    initial_documents = documents

llm_predictor = LLMPredictor(
    llm=OpenAI(
        temperature=0, model_name=PREDICTOR_MODEL_NAME
    )
)

embed_model = OpenAIEmbedding(
    model=EMBEDDING_MODEL_NAME
)

service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    embed_model=embed_model
)

index = GPTQdrantIndex.from_documents(
    client=client, collection_name=COLLECTION_NAME,
    documents=initial_documents, service_context=service_context
)

INFO:httpx:HTTP Request: GET http://localhost:6333/collections/chatgpt_search_collection "HTTP/1.1 404 Not Found"
/Users/ikegamikenshin/Documents/study/chatgpt-practice/.venv/lib/python3.9/site-packages/langchain/llms/openai.py:158: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/ikegamikenshin/Documents/study/chatgpt-practice/.venv/lib/python3.9/site-packages/langchain/llms/openai.py:661: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/chatgpt_search_collection "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: DELETE http://localhost:6333/collections/chatgpt_search_collection "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/c

In [7]:
search_query = "calibrationってどんな技術だっけ？"

response = index.query(
    search_query,
    similarity_top_k=3,
    text_qa_template=CUSTOM_TEXT_QA_PROMPT,
    refine_template=CUSTOM_REFINE_PROMPT
)

print(f"response: {response.response}")

reffer_urls = set([source_node.extra_info["url"] for source_node in response.source_nodes])
for i, url in enumerate(reffer_urls):
    print(f"参照url{i+1}: {url}")

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/chatgpt_search_collection/points/search "HTTP/1.1 200 OK"
INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 17150 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 21 tokens


response: Calibrationは、測定器やモデルの出力値を正確に調整するための技術です。機械学習においては、モデルの確率予測の信頼性を高めるために、Sigmoid/Platt ScaleやIsotonic Regressionなどの手法を用いて、モデルの出力値を各クラスに属する確率に近づけます。また、Calibration Curveを使って、確率予測の信頼度を可視化することもできます。Calibrationの評価指標としては、Brier Scoreがよく使われます。Calibrationは、不均衡データをUndersamplingした場合にも適用できます。ただし、LightGBMや最近のNNは自信過剰であるため、Calibrationが必要かどうかは、train, val, test のそれぞれの予測値/目的変数の平均と分布を見て、総合的に判断する必要があります。また、Calibrationを検討する際には、val / testの予測値の平均値とtrainの目的変数の平均値を見て一致しているかを確認し、train / val / testの予測値の分布を見ることが重要です。
参照url1: https://yukoishizaki.hatenablog.com/entry/2020/05/24/145155


/Users/ikegamikenshin/Documents/study/chatgpt-practice/.venv/lib/python3.9/site-packages/llama_index/data_structs/node_v2.py:181: UserWarning: .extra_info is deprecated, use .node.extra_info instead
  warnings.warn(".extra_info is deprecated, use .node.extra_info instead")


# Documentの追加

In [8]:
client.get_collection(collection_name=COLLECTION_NAME)

INFO:httpx:HTTP Request: GET http://localhost:6333/collections/chatgpt_search_collection "HTTP/1.1 200 OK"


CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=10, indexed_vectors_count=0, points_count=10, segments_count=4, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1536, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None), shard_number=1, replication_factor=1, write_consistency_factor=1, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={})

In [9]:
results, _ = client.scroll(
    collection_name=COLLECTION_NAME,
    scroll_filter=qdrant_client.http.models.Filter(
        must=[
            qdrant_client.http.models.FieldCondition(
                key="extra_info.url",
                match=qdrant_client.http.models.MatchValue(value=ADDITIONAL_URL)
            ),
        ]
    )
)

# 既に登録されているURLの場合はskip
if not results:
    
    additional_urls = [ADDITIONAL_URL]
    additional_documents = SimpleWebPageReader(html_to_text=True).load_data(additional_urls)

    # extra_infoにurlを追加
    for additional_document, url in zip(additional_documents, additional_urls):
        additional_document.extra_info = {"url": url}
    
    # insert
    for additional_document in additional_documents:
        index.insert(additional_document)

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/chatgpt_search_collection/points/scroll "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/chatgpt_search_collection/points?wait=true "HTTP/1.1 200 OK"
INFO:llama_index.token_counter.token_counter:> [insert] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [insert] Total embedding token usage: 13970 tokens


In [10]:
client.get_collection(collection_name=COLLECTION_NAME)

INFO:httpx:HTTP Request: GET http://localhost:6333/collections/chatgpt_search_collection "HTTP/1.1 200 OK"


CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=14, indexed_vectors_count=0, points_count=14, segments_count=4, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1536, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None), shard_number=1, replication_factor=1, write_consistency_factor=1, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={})

In [11]:
search_query = "Qdrantの特徴は?"

response = index.query(
    search_query,
    similarity_top_k=3,
    text_qa_template=CUSTOM_TEXT_QA_PROMPT,
    refine_template=CUSTOM_REFINE_PROMPT
)

print(f"response: {response.response}")

reffer_urls = set([source_node.extra_info["url"] for source_node in response.source_nodes])
for i, url in enumerate(reffer_urls):
    print(f"参照url{i+1}: {url}")

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/chatgpt_search_collection/points/search "HTTP/1.1 200 OK"
INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 15090 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 10 tokens


response: Qdrantの特徴は、オープンソースのRust製ベクトル検索エンジンであり、Python SDK、REST API、gRPCを介してクライアントが接続できること、ベクトル検索においてコサイン類似度を使用していること、大規模なデータセットにも対応できることなどです。Qdrantではコレクションとポイントの概念があり、コレクションはRDBのテーブルに相当し、ポイントはRDBのレコードに相当します。Python SDKを使用してコレクションの作成、ドキュメントの登録、類似ドキュメントの検索が可能であり、絞り込み検索もサポートしています。また、Qdrantを使用したデモサイトも存在し、キーワード検索もサポートしています。ただし、Qdrant自体にはベクトル化の機能は無いため、ドキュメントをベクトル化する必要があります。Qdrantは、Facebook FaissやPynndescentなどのライブラリと比較しても高速であり、大規模なデータセットにも対応できます。
参照url1: https://zenn.dev/tfutada/articles/acf8adbb2ba5be


In [12]:
# indexをローカルに保存
index.save_to_disk("../data/qdrant_index.json")

In [13]:
# index delete用
# client.delete_collection(collection_name=COLLECTION_NAME)